Датасет про грибы, есть категориальные данные
на нем отрабатываю методы AutoML

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
%%time
path = "/kaggle/input/mushroom-classification/mushrooms.csv"
s = !wc -l {path}

print(s)

In [ ]:
df = pd.read_csv(path)
df.info()

In [ ]:
labelEncoder = LabelEncoder()

for col in df.columns:
    df[col] = labelEncoder.fit_transform(df[col])

df = h2o.H2OFrame(df)
df = df.asfactor()

In [ ]:
df.describe ()

In [ ]:
###разделяем выборку на тест и тренировочный датасет
# это не точное значение, тут используются случайные разделения, мат ожидание равно указанным значениям
train, test, valid = df.split_frame ( ratios = [ .7 , .15 ])

In [ ]:
#настройка и  тренировка
y = "class"
x_train = train.columns
x_train.remove(y)
aml = H2OAutoML(max_runtime_secs=150, seed = 1)
aml.train(x = x_train, y = y, training_frame = train)

In [ ]:
#просмотреть результаты работы
lb = aml.leaderboard
lb.head()

In [ ]:
aml.training_info

In [ ]:
test[:10]

In [ ]:
preds = aml.predict(test)
preds

In [ ]:
type(preds)

In [ ]:
print(len(preds))

In [ ]:
df_preds = h2o.as_list(preds)

In [ ]:
type(df_preds)

In [ ]:
df_preds.info()

In [ ]:
df_preds

In [ ]:
#из них получается можно есть:
ones = sum( df_preds['predict']==1 )
ones

In [ ]:
print(ones/1263 * 100) #предсказали столько процентов, что у них будут 1 в тестовом сете, методами AutoML